In [2]:
import netCDF4 as nc
import numpy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import os
import cartopy.feature as cfeature
import glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
#import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cf
import mplotutils as mpu # helper functions for cartopy and matplotlib
#import regionmask
from func_plots import *
from func_stats import *
import geopandas as gpd
import regionmask

import cartopy.crs as ccrs
import cartopy.feature as cf
import mplotutils as mpu # helper functions for cartopy and matplotlib

In [3]:
# Regional plot with coastline and country borders
def format_axes(axes):
    for i, ax in enumerate(axes):
        #ax.set_title(plot_names[i])
        ax.coastlines(resolution='50m', linewidth=0.45)
        ax.add_feature(cf.BORDERS, linewidth=0.3)
        ax.set_extent([lonmin, lonmax, latmin, latmax], crs=data_proj)
        ax.gridlines(draw_labels=False, linewidth=0.3, color="gray", xlocs=range(-180, 180, 10), ylocs=range(-90, 90, 10))

def add_headers(fig, *, row_headers=None, col_headers=None, row_pad=1, col_pad=4, rotate_row_headers=True, **text_kwargs):
    # Based on https://stackoverflow.com/a/25814386
    axes = fig.get_axes()

    for ax in axes:
        sbs = ax.get_subplotspec()

        # Putting headers on cols
        if (col_headers is not None) and sbs.is_first_row():
            ax.annotate(
                col_headers[sbs.colspan.start],
                xy=(0.5, 1),
                xytext=(0, col_pad),
                xycoords="axes fraction",
                textcoords="offset points",
                ha="center",
                va="baseline",
                **text_kwargs,
            )

        # Putting headers on rows
        if (row_headers is not None) and sbs.is_first_col():
            ax.annotate(
                row_headers[sbs.rowspan.start],
                xy=(0, 0.5),
                xytext=(-ax.yaxis.labelpad - row_pad, 0),
                xycoords=ax.transAxes, # ax.yaxis.label replaced by ax.transAxes
                textcoords="offset points",
                ha="center",
                va="center", # vertical even after rotation
                rotation=rotate_row_headers * 90,
                **text_kwargs,
            )

lonmin, lonmax, latmin, latmax = [-11, 37, 35, 70.5]                   # window for plotting
set_lonmin, set_lonmax, set_latmin, set_latmax = [-35, 65, 30, 72.6]   # subset the data to get sensible vmin and vmax for the colorbar
data_proj = ccrs.PlateCarree()
map_proj = ccrs.LambertConformal(central_longitude=15) # for regional maps

In [4]:
with nc.Dataset('/landclim2/yiyaoy/COSMO-CLM2-simulations/timmean/clm5.0_eur0.5_control_h0_2020-2024.nc_timmean') as ds:
    lat = ds.variables['lat'][:]           # read straight into a NumPy array
    lon = ds.variables['lon'][:]
    # shift longitude from [0,360] to [–180,180]
    lon = np.where(lon > 180, lon - 360, lon)

In [9]:
# Metzger et al. 2005 regions ("Environmental Stratification of Europe")
# The sub-regional classification is based on Metzger et al. (2005), aggregated as in Jakob et al. (2014) and in the European chapter of IPCC AR5,
# to represent 5 different climate and environmental characteristics: Alpine, Atlantic, Continental, Northern and Southern Europe
# Load regions and combine
gdf = gpd.read_file('/net/exo/landclim/pesieber/data/Metzger-2005_Env-stratification-Europe/enz_v8.shp').set_index('EnZ_name')
regions = gpd.GeoDataFrame({'region': [], 'geometry': []}, crs=gdf.crs)
regions.loc[0] = {'region': 'Alpine', 'geometry': gdf.loc[['ALN','ALS']].unary_union}
regions.loc[1] = {'region': 'Northern', 'geometry': gdf.loc[['BOR','NEM']].unary_union}
regions.loc[2] = {'region': 'Atlantic', 'geometry': gdf.loc[['ATN','ATC','LUS']].unary_union}
regions.loc[3] = {'region': 'Continental', 'geometry': gdf.loc[['CON','PAN']].unary_union}
regions.loc[4] = {'region': 'Southern', 'geometry': gdf.loc[['MDN','MDS','MDM','ANA']].unary_union}
regions = regions.set_crs('EPSG:3035').to_crs('EPSG:4326')

In [10]:
# Generate a 2D or 3D mask that can be applied to xarray data
mask_3D = regionmask.mask_3D_geopandas(regions, lon, lat) # 3D for groupby
mask_3D = mask_3D.assign_coords(region = regions.region.values) # assign names
#mask_3D.to_netcdf(dpath_proc + 'regionmask_3D_Metzger.nc')
#region_dict = regions.region.to_dict()
mask_regions = regionmask.from_geopandas(regions, names="region", abbrevs="_from_name", name="regionmask") # Regions object
mask_2D = regionmask.mask_geopandas(regions, lon, lat) # 2D for plotting
#mask_2D.to_netcdf(dpath_proc + 'regionmask_2D_Metzger.nc')
regions = mask_3D.region.values

In [26]:
# 1) Map each “scenario” name to its file
FILE_PATHS = {
    "Ctl":        "/landclim2/yiyaoy/COSMO-CLM2-simulations/control/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_control_h0_2025-2059.nc_monmean",
    "Aff":     "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_forest/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_forest_h0_2025-2059.nc_monmean",
    "Def":      "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_grass/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_grass_h0_2025-2059.nc_monmean",
    "Brd":      "/landclim2/yiyaoy/COSMO-CLM2-simulations/only_forest_broadleaf/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_only_forest_broadleaf_h0_2025-2059.nc_monmean",
    "AfB":     "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_forest_broadleaf/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_forest_broadleaf_h0_2025-2059.nc_monmean",
}

def _load_and_region_mean(var, path, mask_2D, region_id):
    """Open TSA, mask out >1e9, apply region mask, then lon→lat means."""
    with nc.Dataset(path) as ds:
        arr = ds.variables[var][:]       # → (time, lat, lon)
    arr = np.squeeze(arr)                 # in case there’s a degenerate dim
    arr[arr > 1e9] = np.nan               # drop fill values

    # set everything *outside* this region to nan
    arr[:, mask_2D != region_id] = np.nan

    # first average over lon (axis=2), then over lat (axis=1)
    return np.nanmean(np.nanmean(arr, axis=2), axis=1)


def get_mean_data_for_plot(var, region_id):
    """
    Returns a dict mapping each scenario to its time‐series of region‐mean TSA.
    E.g. out["control"] is the control run’s 1D array (time down to region‐mean).
    """
    return {
        name: _load_and_region_mean(var, path, mask_2D, region_id)
        for name, path in FILE_PATHS.items()
    }

In [ ]:
Tair_M_0 = get_mean_data_for_plot('TSA', 0)
Tsfc_M_0 = get_mean_data_for_plot('TSKIN', 0)
Tatm_M_0 = get_mean_data_for_plot('TBOT', 0)

Tair_M_1 = get_mean_data_for_plot('TSA', 1)
Tsfc_M_1 = get_mean_data_for_plot('TSKIN', 1)
Tatm_M_1 = get_mean_data_for_plot('TBOT', 1)

Tair_M_2 = get_mean_data_for_plot('TSA', 2)
Tsfc_M_2 = get_mean_data_for_plot('TSKIN', 2)
Tatm_M_2 = get_mean_data_for_plot('TBOT', 2)

Tair_M_3 = get_mean_data_for_plot('TSA', 3)
Tsfc_M_3 = get_mean_data_for_plot('TSKIN', 3)
Tatm_M_3 = get_mean_data_for_plot('TBOT', 3)

Tair_M_4 = get_mean_data_for_plot('TSA', 4)
Tsfc_M_4 = get_mean_data_for_plot('TSKIN', 4)
Tatm_M_4 = get_mean_data_for_plot('TBOT', 4)

In [31]:
# (1) collect your Tair_M dicts
Tair_M_list = [Tair_M_0, Tair_M_1, Tair_M_2, Tair_M_3, Tair_M_4]

# (2) the common keys in each dict
scenarios = ['Ctl','Aff','Def','Brd','AfB']

# Prepare an array: (region_index, scenario_index, month, year)
n_regions = len(Tair_M_list)
n_scenarios = len(scenarios)
n_months = 12
n_years = 35

# one 4‑d array to hold everything
Tair_M_month = np.zeros((n_regions, n_scenarios, n_months, n_years))

# OPTION A: reshape trick (fastest, no inner Python loop)
for i, Tdict in enumerate(Tair_M_list):
    for j, scen in enumerate(scenarios):
        arr = Tdict[scen]   # 1D of length 420 == 12*35
        # reshape into (12 months, 35 years) by filling columns first
        Tair_M_month[i, j] = arr.reshape((n_months, n_years), order='F')

In [48]:
# (1) collect your Tair_M dicts
Tsfc_M_list = [Tsfc_M_0, Tsfc_M_1, Tsfc_M_2, Tsfc_M_3, Tsfc_M_4]

# (2) the common keys in each dict
scenarios = ['Ctl','Aff','Def','Brd','AfB']

# Prepare an array: (region_index, scenario_index, month, year)
n_regions = len(Tsfc_M_list)
n_scenarios = len(scenarios)
n_months = 12
n_years = 35

# one 4‑d array to hold everything
Tsfc_M_month = np.zeros((n_regions, n_scenarios, n_months, n_years))

# OPTION A: reshape trick (fastest, no inner Python loop)
for i, Tdict in enumerate(Tsfc_M_list):
    for j, scen in enumerate(scenarios):
        arr = Tdict[scen]   # 1D of length 420 == 12*35
        # reshape into (12 months, 35 years) by filling columns first
        Tsfc_M_month[i, j] = arr.reshape((n_months, n_years), order='F')

In [49]:
# (1) collect your Tair_M dicts
Tatm_M_list = [Tatm_M_0, Tatm_M_1, Tatm_M_2, Tatm_M_3, Tatm_M_4]

# (2) the common keys in each dict
scenarios = ['Ctl','Aff','Def','Brd','AfB']

# Prepare an array: (region_index, scenario_index, month, year)
n_regions = len(Tatm_M_list)
n_scenarios = len(scenarios)
n_months = 12
n_years = 35

# one 4‑d array to hold everything
Tatm_M_month = np.zeros((n_regions, n_scenarios, n_months, n_years))

# OPTION A: reshape trick (fastest, no inner Python loop)
for i, Tdict in enumerate(Tatm_M_list):
    for j, scen in enumerate(scenarios):
        arr = Tdict[scen]   # 1D of length 420 == 12*35
        # reshape into (12 months, 35 years) by filling columns first
        Tatm_M_month[i, j] = arr.reshape((n_months, n_years), order='F')

In [123]:
import scipy.stats as stats
def get_value(data):
    data_reshape = data.reshape((n_months, n_years), order='F')
    summer_data = data_reshape[5:8, :]
    summer_mean = np.mean(summer_data, axis=0)
    data_mean = np.mean(summer_mean, axis=0)
    std_summer = np.std(summer_mean, ddof=1)
    n = len(summer_mean)
    se = std_summer / np.sqrt(n)
    confidence = 0.95
    t_score = stats.t.ppf((1 + confidence) / 2.0, df=n-1)
    
    return data_mean-273.15, t_score*se

In [147]:
get_value(Tair_N_list[0]['AfB'])

10.201104736328148

In [258]:
data_raw = Tatm_N_list[2]
data = get_value(data_raw['Ctl'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))
data = get_value(data_raw['Brd'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))
data = get_value(data_raw['Def'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))
data = get_value(data_raw['Aff'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))
data = get_value(data_raw['AfB'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))

14.01±0.22
13.98±0.22
14.25±0.23
13.61±0.22
13.56±0.22


In [219]:
data = get_value(Tatm_M_list[1]['Brd'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))

15.25±0.27


In [220]:
data = get_value(Tatm_M_list[1]['Def'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))

15.47±0.27


In [221]:
data = get_value(Tatm_M_list[1]['Aff'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))

15.73±0.28


In [222]:
data = get_value(Tatm_M_list[1]['AfB'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))

15.06±0.26


In [ ]:
def plot_delta_TSA_SHF(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    
    plt.ylim(-2.5,2.5)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)
    # plt.bar(X-0.2, var_exp1_control_month_50, width=0.2, color='tomato', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$')
    # plt.bar(X, var_exp2_control_month_50, width=0.2, color='lightgreen', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$')
    # plt.bar(X+0.2, var_exp3_control_month_50, width=0.2, color='dodgerblue', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$')
    
    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-2.5,2.5)
    plt.xlim(-1,12)
    
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

def plot_delta_TSA_SHF1(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    plt.ylim(-3,3)
    plt.yticks(fontsize=14)
    # plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)

    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-3,3)
    plt.xlim(-1,12)
    # plt.legend(fontsize=14,loc='upper left')
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

fig, axes = plt.subplots(5,2, figsize=(12,20), subplot_kw=dict(projection=map_proj, facecolor="lightgrey"), dpi=300)
fig.subplots_adjust(hspace=0.3, wspace=0.4, left=0.05, right=0.95, top=0.95, bottom=0.05)

X = np.array(range(12))

ax1 = plt.subplot(521, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('a', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_M_list[0]['Brd']-Tsfc_M_list[0]['Ctl'], Tair_M_list[0]['Brd']-Tair_M_list[0]['Ctl'], Tatm_M_list[0]['Brd']-Tatm_M_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
# plot_delta_TSA_SHF(Tair_M_list[0]['Def']-Tair_M_list[0]['Ctl'], Tair_M_list[0]['Brd']-Tair_M_list[0]['Ctl'], 'Def', 'Brd', 'Alpine')
plt.ylim(-1.5,1.5)
plt.legend(fontsize=14,loc='upper left')
ax1.grid()

ax1 = plt.subplot(523, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('c', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_M_list[1]['Brd']-Tsfc_M_list[1]['Ctl'], Tair_M_list[1]['Brd']-Tair_M_list[1]['Ctl'], Tatm_M_list[1]['Brd']-Tatm_M_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
ax1.grid()

ax1 = plt.subplot(525, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('e', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_M_list[2]['Brd']-Tsfc_M_list[2]['Ctl'], Tair_M_list[2]['Brd']-Tair_M_list[2]['Ctl'], Tatm_M_list[2]['Brd']-Tatm_M_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylim(-0.5,0.5)
ax1.grid()
ax1 = plt.subplot(527, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('g', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_M_list[3]['Brd']-Tsfc_M_list[3]['Ctl'], Tair_M_list[3]['Brd']-Tair_M_list[3]['Ctl'], Tatm_M_list[3]['Brd']-Tatm_M_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylim(-0.5,0.5)
ax1.grid()
ax1 = plt.subplot(529, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('i', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_M_list[4]['Brd']-Tsfc_M_list[4]['Ctl'], Tair_M_list[4]['Brd']-Tair_M_list[4]['Ctl'], Tatm_M_list[4]['Brd']-Tatm_M_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylim(-0.5,0.5)

ax1.grid()




ax1 = plt.subplot(522, frameon=True)
# plt.ylabel('$\\Delta$$\mathregular{T_{monM}}$', fontsize=16
plt.title('b', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_M_list[0]['Def']-Tsfc_M_list[0]['Ctl'], Tair_M_list[0]['Def']-Tair_M_list[0]['Ctl'], Tatm_M_list[0]['Def']-Tatm_M_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1.5,1.5)
ax1.grid()
ax1 = plt.subplot(524, frameon=True)
plt.title('d', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_M_list[1]['Def']-Tsfc_M_list[1]['Ctl'], Tair_M_list[1]['Def']-Tair_M_list[1]['Ctl'], Tatm_M_list[1]['Def']-Tatm_M_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)
ax1.grid()
ax1 = plt.subplot(526, frameon=True)
plt.title('f', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_M_list[2]['Def']-Tsfc_M_list[2]['Ctl'], Tair_M_list[2]['Def']-Tair_M_list[2]['Ctl'], Tatm_M_list[2]['Def']-Tatm_M_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-0.5,0.5)
ax1.grid()
ax1 = plt.subplot(528, frameon=True)
plt.title('h', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_M_list[3]['Def']-Tsfc_M_list[3]['Ctl'], Tair_M_list[3]['Def']-Tair_M_list[3]['Ctl'], Tatm_M_list[3]['Def']-Tatm_M_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-0.5,0.5)
ax1.grid()
ax1 = plt.subplot(5,2,10, frameon=True)
plt.title('j', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_M_list[4]['Def']-Tsfc_M_list[4]['Ctl'], Tair_M_list[4]['Def']-Tair_M_list[4]['Ctl'], Tatm_M_list[4]['Def']-Tatm_M_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-0.5,0.5)
ax1.grid()

In [ ]:
def plot_delta_TSA_SHF(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    
    plt.ylim(-2.5,2.5)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)
    # plt.bar(X-0.2, var_exp1_control_month_50, width=0.2, color='tomato', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$')
    # plt.bar(X, var_exp2_control_month_50, width=0.2, color='lightgreen', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$')
    # plt.bar(X+0.2, var_exp3_control_month_50, width=0.2, color='dodgerblue', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$')
    
    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-2.5,2.5)
    plt.xlim(-1,12)
    
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

def plot_delta_TSA_SHF1(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    plt.ylim(-3,3)
    plt.yticks(fontsize=14)
    # plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)

    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-3,3)
    plt.xlim(-1,12)
    # plt.legend(fontsize=14,loc='upper left')
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

fig, axes = plt.subplots(5,2, figsize=(12,20), subplot_kw=dict(projection=map_proj, facecolor="lightgrey"), dpi=300)
fig.subplots_adjust(hspace=0.3, wspace=0.4, left=0.05, right=0.95, top=0.95, bottom=0.05)

X = np.array(range(12))

ax1 = plt.subplot(521, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('a', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_M_list[0]['Aff']-Tsfc_M_list[0]['Ctl'], Tair_M_list[0]['Aff']-Tair_M_list[0]['Ctl'], Tatm_M_list[0]['Aff']-Tatm_M_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
# plot_delta_TSA_SHF(Tair_M_list[0]['Def']-Tair_M_list[0]['Ctl'], Tair_M_list[0]['Brd']-Tair_M_list[0]['Ctl'], 'Def', 'Brd', 'Alpine')
plt.ylim(-1.5,1.5)
plt.legend(fontsize=14,loc='upper right')
ax1.grid()

ax1 = plt.subplot(523, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('c', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_M_list[1]['Aff']-Tsfc_M_list[1]['Ctl'], Tair_M_list[1]['Aff']-Tair_M_list[1]['Ctl'], Tatm_M_list[1]['Aff']-Tatm_M_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
ax1.grid()
plt.ylim(-1.5,1.5)
ax1 = plt.subplot(525, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('e', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_M_list[2]['Aff']-Tsfc_M_list[2]['Ctl'], Tair_M_list[2]['Aff']-Tair_M_list[2]['Ctl'], Tatm_M_list[2]['Aff']-Tatm_M_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylim(-1,1)
ax1.grid()
ax1 = plt.subplot(527, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('g', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_M_list[3]['Aff']-Tsfc_M_list[3]['Ctl'], Tair_M_list[3]['Aff']-Tair_M_list[3]['Ctl'], Tatm_M_list[3]['Aff']-Tatm_M_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylim(-1.5,1.5)
ax1.grid()
ax1 = plt.subplot(529, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('i', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_M_list[4]['Aff']-Tsfc_M_list[4]['Ctl'], Tair_M_list[4]['Aff']-Tair_M_list[4]['Ctl'], Tatm_M_list[4]['Aff']-Tatm_M_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylim(-1.5,1.5)

ax1.grid()




ax1 = plt.subplot(522, frameon=True)
# plt.ylabel('$\\Delta$$\mathregular{T_{monM}}$', fontsize=16
plt.title('b', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_M_list[0]['AfB']-Tsfc_M_list[0]['Ctl'], Tair_M_list[0]['AfB']-Tair_M_list[0]['Ctl'], Tatm_M_list[0]['AfB']-Tatm_M_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1.5,1.5)
ax1.grid()
ax1 = plt.subplot(524, frameon=True)
plt.title('d', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_M_list[1]['AfB']-Tsfc_M_list[1]['Ctl'], Tair_M_list[1]['AfB']-Tair_M_list[1]['Ctl'], Tatm_M_list[1]['AfB']-Tatm_M_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)
ax1.grid()
plt.ylim(-1.5,1.5)
ax1 = plt.subplot(526, frameon=True)
plt.title('f', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_M_list[2]['AfB']-Tsfc_M_list[2]['Ctl'], Tair_M_list[2]['AfB']-Tair_M_list[2]['Ctl'], Tatm_M_list[2]['AfB']-Tatm_M_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1,1)
ax1.grid()
ax1 = plt.subplot(528, frameon=True)
plt.title('h', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_M_list[3]['AfB']-Tsfc_M_list[3]['Ctl'], Tair_M_list[3]['AfB']-Tair_M_list[3]['Ctl'], Tatm_M_list[3]['AfB']-Tatm_M_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1.5,1.5)
ax1.grid()
ax1 = plt.subplot(5,2,10, frameon=True)
plt.title('j', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_M_list[4]['AfB']-Tsfc_M_list[4]['Ctl'], Tair_M_list[4]['AfB']-Tair_M_list[4]['Ctl'], Tatm_M_list[4]['AfB']-Tatm_M_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monM}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1.5,1.5)
ax1.grid()

In [54]:
# 1) Map each “scenario” name to its file
FILE_PATHS_MAX = {
    "Ctl":        "/landclim2/yiyaoy/COSMO-CLM2-simulations/control/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_control_h2_2025-2059.nc_monmean",
    "Aff":     "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_forest/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_forest_h2_2025-2059.nc_monmean",
    "Def":      "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_grass/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_grass_h2_2025-2059.nc_monmean",
    "Brd":      "/landclim2/yiyaoy/COSMO-CLM2-simulations/only_forest_broadleaf/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_only_forest_broadleaf_h2_2025-2059.nc_monmean",
    "AfB":     "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_forest_broadleaf/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_forest_broadleaf_h2_2025-2059.nc_monmean",
}

def _load_and_region_max(var, path, mask_2D, region_id):
    """Open TSA, mask out >1e9, apply region mask, then lon→lat means."""
    with nc.Dataset(path) as ds:
        arr = ds.variables[var][:]       # → (time, lat, lon)
    arr = np.squeeze(arr)                 # in case there’s a degenerate dim
    arr[arr > 1e9] = np.nan               # drop fill values

    # set everything *outside* this region to nan
    arr[:, mask_2D != region_id] = np.nan

    # first average over lon (axis=2), then over lat (axis=1)
    return np.nanmean(np.nanmean(arr, axis=2), axis=1)


def get_max_data_for_plot(var, region_id):
    """
    Returns a dict mapping each scenario to its time‐series of region‐mean TSA.
    E.g. out["control"] is the control run’s 1D array (time down to region‐mean).
    """
    return {
        name: _load_and_region_max(var, path, mask_2D, region_id)
        for name, path in FILE_PATHS_MAX.items()
    }

In [ ]:
Tair_X_0 = get_max_data_for_plot('TSA', 0)
Tsfc_X_0 = get_max_data_for_plot('TSKIN', 0)
Tatm_X_0 = get_max_data_for_plot('TBOT', 0)

Tair_X_1 = get_max_data_for_plot('TSA', 1)
Tsfc_X_1 = get_max_data_for_plot('TSKIN', 1)
Tatm_X_1 = get_max_data_for_plot('TBOT', 1)

Tair_X_2 = get_max_data_for_plot('TSA', 2)
Tsfc_X_2 = get_max_data_for_plot('TSKIN', 2)
Tatm_X_2 = get_max_data_for_plot('TBOT', 2)

Tair_X_3 = get_max_data_for_plot('TSA', 3)
Tsfc_X_3 = get_max_data_for_plot('TSKIN', 3)
Tatm_X_3 = get_max_data_for_plot('TBOT', 3)

Tair_X_4 = get_max_data_for_plot('TSA', 4)
Tsfc_X_4 = get_max_data_for_plot('TSKIN', 4)
Tatm_X_4 = get_max_data_for_plot('TBOT', 4)

In [79]:
# (1) collect your Tair_M dicts
Tair_X_list = [Tair_X_0, Tair_X_1, Tair_X_2, Tair_X_3, Tair_X_4]

# (2) the common keys in each dict
scenarios = ['Ctl','Aff','Def','Brd','AfB']

# Prepare an array: (region_index, scenario_index, month, year)
n_regions = len(Tair_X_list)
n_scenarios = len(scenarios)
n_months = 12
n_years = 35

# one 4‑d array to hold everything
Tair_X_month = np.zeros((n_regions, n_scenarios, n_months, n_years))

# OPTION A: reshape trick (fastest, no inner Python loop)
for i, Tdict in enumerate(Tair_X_list):
    for j, scen in enumerate(scenarios):
        arr = Tdict[scen]   # 1D of length 420 == 12*35
        # reshape into (12 months, 35 years) by filling columns first
        Tair_X_month[i, j] = arr.reshape((n_months, n_years), order='F')

In [81]:
# (1) collect your Tair_M dicts
Tsfc_X_list = [Tsfc_X_0, Tsfc_X_1, Tsfc_X_2, Tsfc_X_3, Tsfc_X_4]

# (2) the common keys in each dict
scenarios = ['Ctl','Aff','Def','Brd','AfB']

# Prepare an array: (region_index, scenario_index, month, year)
n_regions = len(Tsfc_X_list)
n_scenarios = len(scenarios)
n_months = 12
n_years = 35

# one 4‑d array to hold everything
Tsfc_X_month = np.zeros((n_regions, n_scenarios, n_months, n_years))

# OPTION A: reshape trick (fastest, no inner Python loop)
for i, Tdict in enumerate(Tsfc_X_list):
    for j, scen in enumerate(scenarios):
        arr = Tdict[scen]   # 1D of length 420 == 12*35
        # reshape into (12 months, 35 years) by filling columns first
        Tsfc_X_month[i, j] = arr.reshape((n_months, n_years), order='F')

In [82]:
# (1) collect your Tair_M dicts
Tatm_X_list = [Tatm_X_0, Tatm_X_1, Tatm_X_2, Tatm_X_3, Tatm_X_4]

# (2) the common keys in each dict
scenarios = ['Ctl','Aff','Def','Brd','AfB']

# Prepare an array: (region_index, scenario_index, month, year)
n_regions = len(Tatm_X_list)
n_scenarios = len(scenarios)
n_months = 12
n_years = 35

# one 4‑d array to hold everything
Tatm_X_month = np.zeros((n_regions, n_scenarios, n_months, n_years))

# OPTION A: reshape trick (fastest, no inner Python loop)
for i, Tdict in enumerate(Tatm_X_list):
    for j, scen in enumerate(scenarios):
        arr = Tdict[scen]   # 1D of length 420 == 12*35
        # reshape into (12 months, 35 years) by filling columns first
        Tatm_X_month[i, j] = arr.reshape((n_months, n_years), order='F')

In [ ]:
def plot_delta_TSA_SHF(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    
    plt.ylim(-2.5,2.5)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)
    # plt.bar(X-0.2, var_exp1_control_month_50, width=0.2, color='tomato', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$')
    # plt.bar(X, var_exp2_control_month_50, width=0.2, color='lightgreen', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$')
    # plt.bar(X+0.2, var_exp3_control_month_50, width=0.2, color='dodgerblue', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$')
    
    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-2.5,2.5)
    plt.xlim(-1,12)
    
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

def plot_delta_TSA_SHF1(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    plt.ylim(-3,3)
    plt.yticks(fontsize=14)
    # plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)

    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-3,3)
    plt.xlim(-1,12)
    # plt.legend(fontsize=14,loc='upper left')
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

fig, axes = plt.subplots(5,2, figsize=(12,20), subplot_kw=dict(projection=map_proj, facecolor="lightgrey"), dpi=300)
fig.subplots_adjust(hspace=0.3, wspace=0.4, left=0.05, right=0.95, top=0.95, bottom=0.05)

X = np.array(range(12))

ax1 = plt.subplot(521, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('a', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_X_list[0]['Brd']-Tsfc_X_list[0]['Ctl'], Tair_X_list[0]['Brd']-Tair_X_list[0]['Ctl'], Tatm_X_list[0]['Brd']-Tatm_X_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
# plot_delta_TSA_SHF(Tair_X_list[0]['Def']-Tair_X_list[0]['Ctl'], Tair_X_list[0]['Brd']-Tair_X_list[0]['Ctl'], 'Def', 'Brd', 'Alpine')
plt.ylim(-1.5,1.5)
plt.legend(fontsize=14,loc='upper left')
ax1.grid()

ax1 = plt.subplot(523, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('c', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_X_list[1]['Brd']-Tsfc_X_list[1]['Ctl'], Tair_X_list[1]['Brd']-Tair_X_list[1]['Ctl'], Tatm_X_list[1]['Brd']-Tatm_X_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
plt.ylim(-3.5,3.5)
ax1.grid()

ax1 = plt.subplot(525, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('e', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_X_list[2]['Brd']-Tsfc_X_list[2]['Ctl'], Tair_X_list[2]['Brd']-Tair_X_list[2]['Ctl'], Tatm_X_list[2]['Brd']-Tatm_X_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylim(-1,1)
ax1.grid()
ax1 = plt.subplot(527, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('g', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_X_list[3]['Brd']-Tsfc_X_list[3]['Ctl'], Tair_X_list[3]['Brd']-Tair_X_list[3]['Ctl'], Tatm_X_list[3]['Brd']-Tatm_X_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylim(-1,1)
ax1.grid()
ax1 = plt.subplot(529, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('i', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_X_list[4]['Brd']-Tsfc_X_list[4]['Ctl'], Tair_X_list[4]['Brd']-Tair_X_list[4]['Ctl'], Tatm_X_list[4]['Brd']-Tatm_X_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylim(-1,1)

ax1.grid()




ax1 = plt.subplot(522, frameon=True)
# plt.ylabel('$\\Delta$$\mathregular{T_{monX}}$', fontsize=16
plt.title('b', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_X_list[0]['Def']-Tsfc_X_list[0]['Ctl'], Tair_X_list[0]['Def']-Tair_X_list[0]['Ctl'], Tatm_X_list[0]['Def']-Tatm_X_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1.5,1.5)
ax1.grid()
ax1 = plt.subplot(524, frameon=True)
plt.title('d', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_X_list[1]['Def']-Tsfc_X_list[1]['Ctl'], Tair_X_list[1]['Def']-Tair_X_list[1]['Ctl'], Tatm_X_list[1]['Def']-Tatm_X_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-3.5,3.5)
ax1.grid()
ax1 = plt.subplot(526, frameon=True)
plt.title('f', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_X_list[2]['Def']-Tsfc_X_list[2]['Ctl'], Tair_X_list[2]['Def']-Tair_X_list[2]['Ctl'], Tatm_X_list[2]['Def']-Tatm_X_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1,1)
ax1.grid()
ax1 = plt.subplot(528, frameon=True)
plt.title('h', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_X_list[3]['Def']-Tsfc_X_list[3]['Ctl'], Tair_X_list[3]['Def']-Tair_X_list[3]['Ctl'], Tatm_X_list[3]['Def']-Tatm_X_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1,1)
ax1.grid()
ax1 = plt.subplot(5,2,10, frameon=True)
plt.title('j', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_X_list[4]['Def']-Tsfc_X_list[4]['Ctl'], Tair_X_list[4]['Def']-Tair_X_list[4]['Ctl'], Tatm_X_list[4]['Def']-Tatm_X_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1,1)
ax1.grid()

In [ ]:
def plot_delta_TSA_SHF(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    
    plt.ylim(-2.5,2.5)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)
    # plt.bar(X-0.2, var_exp1_control_month_50, width=0.2, color='tomato', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$')
    # plt.bar(X, var_exp2_control_month_50, width=0.2, color='lightgreen', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$')
    # plt.bar(X+0.2, var_exp3_control_month_50, width=0.2, color='dodgerblue', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$')
    
    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-2.5,2.5)
    plt.xlim(-1,12)
    
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

def plot_delta_TSA_SHF1(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    plt.ylim(-3,3)
    plt.yticks(fontsize=14)
    # plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)

    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-3,3)
    plt.xlim(-1,12)
    # plt.legend(fontsize=14,loc='upper left')
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

fig, axes = plt.subplots(5,2, figsize=(12,20), subplot_kw=dict(projection=map_proj, facecolor="lightgrey"), dpi=300)
fig.subplots_adjust(hspace=0.3, wspace=0.4, left=0.05, right=0.95, top=0.95, bottom=0.05)

X = np.array(range(12))

ax1 = plt.subplot(521, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('a', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_X_list[0]['Aff']-Tsfc_X_list[0]['Ctl'], Tair_X_list[0]['Aff']-Tair_X_list[0]['Ctl'], Tatm_X_list[0]['Aff']-Tatm_X_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
# plot_delta_TSA_SHF(Tair_X_list[0]['Def']-Tair_X_list[0]['Ctl'], Tair_X_list[0]['Brd']-Tair_X_list[0]['Ctl'], 'Def', 'Brd', 'Alpine')
plt.ylim(-2.5,2.5)
plt.legend(fontsize=14,loc='upper right')
ax1.grid()

ax1 = plt.subplot(523, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('c', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_X_list[1]['Aff']-Tsfc_X_list[1]['Ctl'], Tair_X_list[1]['Aff']-Tair_X_list[1]['Ctl'], Tatm_X_list[1]['Aff']-Tatm_X_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
plt.ylim(-2,2)
ax1.grid()

ax1 = plt.subplot(525, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('e', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_X_list[2]['Aff']-Tsfc_X_list[2]['Ctl'], Tair_X_list[2]['Aff']-Tair_X_list[2]['Ctl'], Tatm_X_list[2]['Aff']-Tatm_X_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylim(-2,2)
ax1.grid()
ax1 = plt.subplot(527, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('g', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_X_list[3]['Aff']-Tsfc_X_list[3]['Ctl'], Tair_X_list[3]['Aff']-Tair_X_list[3]['Ctl'], Tatm_X_list[3]['Aff']-Tatm_X_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylim(-2.5,2.5)
ax1.grid()
ax1 = plt.subplot(529, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('i', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_X_list[4]['Aff']-Tsfc_X_list[4]['Ctl'], Tair_X_list[4]['Aff']-Tair_X_list[4]['Ctl'], Tatm_X_list[4]['Aff']-Tatm_X_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylim(-2,2)

ax1.grid()




ax1 = plt.subplot(522, frameon=True)
# plt.ylabel('$\\Delta$$\mathregular{T_{monX}}$', fontsize=16
plt.title('b', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_X_list[0]['AfB']-Tsfc_X_list[0]['Ctl'], Tair_X_list[0]['AfB']-Tair_X_list[0]['Ctl'], Tatm_X_list[0]['AfB']-Tatm_X_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-2.5,2.5)
ax1.grid()
ax1 = plt.subplot(524, frameon=True)
plt.title('d', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_X_list[1]['AfB']-Tsfc_X_list[1]['Ctl'], Tair_X_list[1]['AfB']-Tair_X_list[1]['Ctl'], Tatm_X_list[1]['AfB']-Tatm_X_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-2,2)
ax1.grid()
ax1 = plt.subplot(526, frameon=True)
plt.title('f', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_X_list[2]['AfB']-Tsfc_X_list[2]['Ctl'], Tair_X_list[2]['AfB']-Tair_X_list[2]['Ctl'], Tatm_X_list[2]['AfB']-Tatm_X_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-2,2)
ax1.grid()
ax1 = plt.subplot(528, frameon=True)
plt.title('h', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_X_list[3]['AfB']-Tsfc_X_list[3]['Ctl'], Tair_X_list[3]['AfB']-Tair_X_list[3]['Ctl'], Tatm_X_list[3]['AfB']-Tatm_X_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-2.5,2.5)
ax1.grid()
ax1 = plt.subplot(5,2,10, frameon=True)
plt.title('j', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_X_list[4]['AfB']-Tsfc_X_list[4]['Ctl'], Tair_X_list[4]['AfB']-Tair_X_list[4]['Ctl'], Tatm_X_list[4]['AfB']-Tatm_X_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monX}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-2,2)
ax1.grid()

In [58]:
# 1) Map each “scenario” name to its file
FILE_PATHS_MIN = {
    "Ctl":        "/landclim2/yiyaoy/COSMO-CLM2-simulations/control/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_control_h4_2025-2059.nc_monmean",
    "Aff":     "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_forest/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_forest_h4_2025-2059.nc_monmean",
    "Def":      "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_grass/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_grass_h4_2025-2059.nc_monmean",
    "Brd":      "/landclim2/yiyaoy/COSMO-CLM2-simulations/only_forest_broadleaf/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_only_forest_broadleaf_h4_2025-2059.nc_monmean",
    "AfB":     "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_forest_broadleaf/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_forest_broadleaf_h4_2025-2059.nc_monmean",
}

def _load_and_region_min(var, path, mask_2D, region_id):
    """Open TSA, mask out >1e9, apply region mask, then lon→lat means."""
    with nc.Dataset(path) as ds:
        arr = ds.variables[var][:]       # → (time, lat, lon)
    arr = np.squeeze(arr)                 # in case there’s a degenerate dim
    arr[arr > 1e9] = np.nan               # drop fill values

    # set everything *outside* this region to nan
    arr[:, mask_2D != region_id] = np.nan

    # first average over lon (axis=2), then over lat (axis=1)
    return np.nanmean(np.nanmean(arr, axis=2), axis=1)


def get_min_data_for_plot(var, region_id):
    """
    Returns a dict mapping each scenario to its time‐series of region‐mean TSA.
    E.g. out["control"] is the control run’s 1D array (time down to region‐mean).
    """
    return {
        name: _load_and_region_min(var, path, mask_2D, region_id)
        for name, path in FILE_PATHS_MIN.items()
    }

In [ ]:
Tair_N_0 = get_min_data_for_plot('TSA', 0)
Tsfc_N_0 = get_min_data_for_plot('TSKIN', 0)
Tatm_N_0 = get_min_data_for_plot('TBOT', 0)

Tair_N_1 = get_min_data_for_plot('TSA', 1)
Tsfc_N_1 = get_min_data_for_plot('TSKIN', 1)
Tatm_N_1 = get_min_data_for_plot('TBOT', 1)

Tair_N_2 = get_min_data_for_plot('TSA', 2)
Tsfc_N_2 = get_min_data_for_plot('TSKIN', 2)
Tatm_N_2 = get_min_data_for_plot('TBOT', 2)

Tair_N_3 = get_min_data_for_plot('TSA', 3)
Tsfc_N_3 = get_min_data_for_plot('TSKIN', 3)
Tatm_N_3 = get_min_data_for_plot('TBOT', 3)

Tair_N_4 = get_min_data_for_plot('TSA', 4)
Tsfc_N_4 = get_min_data_for_plot('TSKIN', 4)
Tatm_N_4 = get_min_data_for_plot('TBOT', 4)

In [95]:
# (1) collect your Tair_M dicts
Tatm_N_list = [Tatm_N_0, Tatm_N_1, Tatm_N_2, Tatm_N_3, Tatm_N_4]

# (2) the common keys in each dict
scenarios = ['Ctl','Aff','Def','Brd','AfB']

# Prepare an array: (region_index, scenario_index, month, year)
n_regions = len(Tatm_N_list)
n_scenarios = len(scenarios)
n_months = 12
n_years = 35

# one 4‑d array to hold everything
Tatm_N_month = np.zeros((n_regions, n_scenarios, n_months, n_years))

# OPTION A: reshape trick (fastest, no inner Python loop)
for i, Tdict in enumerate(Tatm_N_list):
    for j, scen in enumerate(scenarios):
        arr = Tdict[scen]   # 1D of length 420 == 12*35
        # reshape into (12 months, 35 years) by filling columns first
        Tatm_N_month[i, j] = arr.reshape((n_months, n_years), order='F')

In [97]:
# (1) collect your Tair_M dicts
Tair_N_list = [Tair_N_0, Tair_N_1, Tair_N_2, Tair_N_3, Tair_N_4]

# (2) the common keys in each dict
scenarios = ['Ctl','Aff','Def','Brd','AfB']

# Prepare an array: (region_index, scenario_index, month, year)
n_regions = len(Tair_N_list)
n_scenarios = len(scenarios)
n_months = 12
n_years = 35

# one 4‑d array to hold everything
Tair_N_month = np.zeros((n_regions, n_scenarios, n_months, n_years))

# OPTION A: reshape trick (fastest, no inner Python loop)
for i, Tdict in enumerate(Tair_N_list):
    for j, scen in enumerate(scenarios):
        arr = Tdict[scen]   # 1D of length 420 == 12*35
        # reshape into (12 months, 35 years) by filling columns first
        Tair_N_month[i, j] = arr.reshape((n_months, n_years), order='F')

In [98]:
# (1) collect your Tsfc_M dicts
Tsfc_N_list = [Tsfc_N_0, Tsfc_N_1, Tsfc_N_2, Tsfc_N_3, Tsfc_N_4]

# (2) the common keys in each dict
scenarios = ['Ctl','Aff','Def','Brd','AfB']

# Prepare an array: (region_index, scenario_index, month, year)
n_regions = len(Tsfc_N_list)
n_scenarios = len(scenarios)
n_months = 12
n_years = 35

# one 4‑d array to hold everything
Tsfc_N_month = np.zeros((n_regions, n_scenarios, n_months, n_years))

# OPTION A: reshape trick (fastest, no inner Python loop)
for i, Tdict in enumerate(Tsfc_N_list):
    for j, scen in enumerate(scenarios):
        arr = Tdict[scen]   # 1D of length 420 == 12*35
        # reshape into (12 months, 35 years) by filling columns first
        Tsfc_N_month[i, j] = arr.reshape((n_months, n_years), order='F')

In [ ]:
def plot_delta_TSA_SHF(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    
    plt.ylim(-2.5,2.5)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)
    # plt.bar(X-0.2, var_exp1_control_month_50, width=0.2, color='tomato', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$')
    # plt.bar(X, var_exp2_control_month_50, width=0.2, color='lightgreen', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$')
    # plt.bar(X+0.2, var_exp3_control_month_50, width=0.2, color='dodgerblue', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$')
    
    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-2.5,2.5)
    plt.xlim(-1,12)
    
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

def plot_delta_TSA_SHF1(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    plt.ylim(-3,3)
    plt.yticks(fontsize=14)
    # plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)

    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-3,3)
    plt.xlim(-1,12)
    # plt.legend(fontsize=14,loc='upper left')
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

fig, axes = plt.subplots(5,2, figsize=(12,20), subplot_kw=dict(projection=map_proj, facecolor="lightgrey"), dpi=300)
fig.subplots_adjust(hspace=0.3, wspace=0.4, left=0.05, right=0.95, top=0.95, bottom=0.05)

X = np.array(range(12))

ax1 = plt.subplot(521, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('a', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_N_list[0]['Brd']-Tsfc_N_list[0]['Ctl'], Tair_N_list[0]['Brd']-Tair_N_list[0]['Ctl'], Tatm_N_list[0]['Brd']-Tatm_N_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
# plot_delta_TSA_SHF(Tair_N_list[0]['Def']-Tair_N_list[0]['Ctl'], Tair_N_list[0]['Brd']-Tair_N_list[0]['Ctl'], 'Def', 'Brd', 'Alpine')
plt.ylim(-1.5,1.5)
plt.legend(fontsize=14,loc='upper left')
ax1.grid()

ax1 = plt.subplot(523, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('c', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_N_list[1]['Brd']-Tsfc_N_list[1]['Ctl'], Tair_N_list[1]['Brd']-Tair_N_list[1]['Ctl'], Tatm_N_list[1]['Brd']-Tatm_N_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
ax1.grid()

ax1 = plt.subplot(525, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('e', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_N_list[2]['Brd']-Tsfc_N_list[2]['Ctl'], Tair_N_list[2]['Brd']-Tair_N_list[2]['Ctl'], Tatm_N_list[2]['Brd']-Tatm_N_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylim(-0.5,0.5)
ax1.grid()
ax1 = plt.subplot(527, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('g', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_N_list[3]['Brd']-Tsfc_N_list[3]['Ctl'], Tair_N_list[3]['Brd']-Tair_N_list[3]['Ctl'], Tatm_N_list[3]['Brd']-Tatm_N_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylim(-0.5,0.5)
ax1.grid()
ax1 = plt.subplot(529, frameon=True)
plt.ylabel('Brd\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('i', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_N_list[4]['Brd']-Tsfc_N_list[4]['Ctl'], Tair_N_list[4]['Brd']-Tair_N_list[4]['Ctl'], Tatm_N_list[4]['Brd']-Tatm_N_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylim(-0.5,0.5)

ax1.grid()




ax1 = plt.subplot(522, frameon=True)
# plt.ylabel('$\\Delta$$\mathregular{T_{monN}}$', fontsize=16
plt.title('b', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_N_list[0]['Def']-Tsfc_N_list[0]['Ctl'], Tair_N_list[0]['Def']-Tair_N_list[0]['Ctl'], Tatm_N_list[0]['Def']-Tatm_N_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1.5,1.5)
ax1.grid()
ax1 = plt.subplot(524, frameon=True)
plt.title('d', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_N_list[1]['Def']-Tsfc_N_list[1]['Ctl'], Tair_N_list[1]['Def']-Tair_N_list[1]['Ctl'], Tatm_N_list[1]['Def']-Tatm_N_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)
ax1.grid()
ax1 = plt.subplot(526, frameon=True)
plt.title('f', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_N_list[2]['Def']-Tsfc_N_list[2]['Ctl'], Tair_N_list[2]['Def']-Tair_N_list[2]['Ctl'], Tatm_N_list[2]['Def']-Tatm_N_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-0.5,0.5)
ax1.grid()
ax1 = plt.subplot(528, frameon=True)
plt.title('h', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_N_list[3]['Def']-Tsfc_N_list[3]['Ctl'], Tair_N_list[3]['Def']-Tair_N_list[3]['Ctl'], Tatm_N_list[3]['Def']-Tatm_N_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-0.5,0.5)
ax1.grid()
ax1 = plt.subplot(5,2,10, frameon=True)
plt.title('j', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_N_list[4]['Def']-Tsfc_N_list[4]['Ctl'], Tair_N_list[4]['Def']-Tair_N_list[4]['Ctl'], Tatm_N_list[4]['Def']-Tatm_N_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylabel('Def\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-0.5,0.5)
ax1.grid()

In [ ]:
def plot_delta_TSA_SHF(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    
    plt.ylim(-2.5,2.5)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)
    # plt.bar(X-0.2, var_exp1_control_month_50, width=0.2, color='tomato', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$')
    # plt.bar(X, var_exp2_control_month_50, width=0.2, color='lightgreen', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$')
    # plt.bar(X+0.2, var_exp3_control_month_50, width=0.2, color='dodgerblue', alpha=0.8, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$')
    
    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-2.5,2.5)
    plt.xlim(-1,12)
    
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

def plot_delta_TSA_SHF1(var_exp1_control_month, var_exp2_control_month, var_exp3_control_month, str_1, str_2, str_3, region_name):

    var_exp1_control_month_50 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp1_control_month_25 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp1_control_month_75 = np.percentile(var_exp1_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    var_exp2_control_month_50 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp2_control_month_25 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp2_control_month_75 = np.percentile(var_exp2_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)

    var_exp3_control_month_50 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 50, axis=1)
    var_exp3_control_month_25 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 25, axis=1)
    var_exp3_control_month_75 = np.percentile(var_exp3_control_month.reshape((n_months, n_years), order='F'), 75, axis=1)
    
    plt.ylim(-3,3)
    plt.yticks(fontsize=14)
    # plt.legend(fontsize=10,loc='lower right')

    ax1.set_title(region_name, fontsize=16)

    plt.errorbar(X-0.2, var_exp1_control_month_50, yerr=[abs(var_exp1_control_month_50-var_exp1_control_month_25), abs(var_exp1_control_month_75-var_exp1_control_month_50)], color='brown', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_1+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X, var_exp2_control_month_50, yerr=[abs(var_exp2_control_month_50-var_exp2_control_month_25), abs(var_exp2_control_month_75-var_exp2_control_month_50)], color='green', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_2+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.errorbar(X+0.2, var_exp3_control_month_50, yerr=[abs(var_exp3_control_month_50-var_exp3_control_month_25), abs(var_exp3_control_month_75-var_exp3_control_month_50)], color='dodgerblue', fmt='-',capsize=3, label='$\\Delta$$\mathregular{T_{'+str_3+'}}$',elinewidth=1,capthick=1, alpha=0.8)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    plt.ylim(-3,3)
    plt.xlim(-1,12)
    # plt.legend(fontsize=14,loc='upper left')
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=14)
    plt.yticks(fontsize=14)

fig, axes = plt.subplots(5,2, figsize=(12,20), subplot_kw=dict(projection=map_proj, facecolor="lightgrey"), dpi=300)
fig.subplots_adjust(hspace=0.3, wspace=0.4, left=0.05, right=0.95, top=0.95, bottom=0.05)

X = np.array(range(12))

ax1 = plt.subplot(521, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('a', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_N_list[0]['Aff']-Tsfc_N_list[0]['Ctl'], Tair_N_list[0]['Aff']-Tair_N_list[0]['Ctl'], Tatm_N_list[0]['Aff']-Tatm_N_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
# plot_delta_TSA_SHF(Tair_N_list[0]['Def']-Tair_N_list[0]['Ctl'], Tair_N_list[0]['Brd']-Tair_N_list[0]['Ctl'], 'Def', 'Brd', 'Alpine')
plt.ylim(-1.5,1.5)
plt.legend(fontsize=14,loc='upper right')
ax1.grid()

ax1 = plt.subplot(523, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('c', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_N_list[1]['Aff']-Tsfc_N_list[1]['Ctl'], Tair_N_list[1]['Aff']-Tair_N_list[1]['Ctl'], Tatm_N_list[1]['Aff']-Tatm_N_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
plt.ylim(-1,1)
ax1.grid()

ax1 = plt.subplot(525, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('e', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_N_list[2]['Aff']-Tsfc_N_list[2]['Ctl'], Tair_N_list[2]['Aff']-Tair_N_list[2]['Ctl'], Tatm_N_list[2]['Aff']-Tatm_N_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylim(-1,1)
ax1.grid()
ax1 = plt.subplot(527, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('g', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_N_list[3]['Aff']-Tsfc_N_list[3]['Ctl'], Tair_N_list[3]['Aff']-Tair_N_list[3]['Ctl'], Tatm_N_list[3]['Aff']-Tatm_N_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylim(-1,1)
ax1.grid()
ax1 = plt.subplot(529, frameon=True)
plt.ylabel('Aff\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)

plt.title('i', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(Tsfc_N_list[4]['Aff']-Tsfc_N_list[4]['Ctl'], Tair_N_list[4]['Aff']-Tair_N_list[4]['Ctl'], Tatm_N_list[4]['Aff']-Tatm_N_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylim(-1,1)

ax1.grid()




ax1 = plt.subplot(522, frameon=True)
# plt.ylabel('$\\Delta$$\mathregular{T_{monN}}$', fontsize=16
plt.title('b', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_N_list[0]['AfB']-Tsfc_N_list[0]['Ctl'], Tair_N_list[0]['AfB']-Tair_N_list[0]['Ctl'], Tatm_N_list[0]['AfB']-Tatm_N_list[0]['Ctl'], 'sfc', 'air', 'atm', 'Alpine')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1.5,1.5)
ax1.grid()
ax1 = plt.subplot(524, frameon=True)
plt.title('d', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_N_list[1]['AfB']-Tsfc_N_list[1]['Ctl'], Tair_N_list[1]['AfB']-Tair_N_list[1]['Ctl'], Tatm_N_list[1]['AfB']-Tatm_N_list[1]['Ctl'], 'sfc', 'air', 'atm', 'Northern')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1,1)
ax1.grid()
ax1 = plt.subplot(526, frameon=True)
plt.title('f', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_N_list[2]['AfB']-Tsfc_N_list[2]['Ctl'], Tair_N_list[2]['AfB']-Tair_N_list[2]['Ctl'], Tatm_N_list[2]['AfB']-Tatm_N_list[2]['Ctl'], 'sfc', 'air', 'atm', 'Atlantic')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1,1)
ax1.grid()
ax1 = plt.subplot(528, frameon=True)
plt.title('h', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_N_list[3]['AfB']-Tsfc_N_list[3]['Ctl'], Tair_N_list[3]['AfB']-Tair_N_list[3]['Ctl'], Tatm_N_list[3]['AfB']-Tatm_N_list[3]['Ctl'], 'sfc', 'air', 'atm', 'Continental')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1,1)
ax1.grid()
ax1 = plt.subplot(5,2,10, frameon=True)
plt.title('j', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF1(Tsfc_N_list[4]['AfB']-Tsfc_N_list[4]['Ctl'], Tair_N_list[4]['AfB']-Tair_N_list[4]['Ctl'], Tatm_N_list[4]['AfB']-Tatm_N_list[4]['Ctl'], 'sfc', 'air', 'atm', 'Southern')
plt.ylabel('AfB\N{MINUS SIGN}Ctl $\\Delta$$\mathregular{T_{monN}}$($\mathregular{^\circ C}$)', fontsize=18)
plt.ylim(-1,1)
ax1.grid()